In [ ]:
# %pip install ipympl

In [ ]:
%matplotlib widget

import caiman as cm 
import matplotlib.pyplot as plt
import numpy as np
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf.params import CNMFParams 
from caiman.source_extraction.cnmf import cnmf
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour, view_quilt, inspect_correlation_pnr, nb_inspect_correlation_pnr 
import sciebo

In [ ]:
sciebo.download_file_from_sciebo('https://uni-bonn.sciebo.de/s/RR7qj7tklW1rX25', 'data', 'data_endoscope.tif')

# Constrained Non-Negative Matrix Factorization- Endoscopic (CNMF-E) Algorithm

Usually for 1-photon (or 2-photon with ill behaved background activity), we should employ the workflow outlined in this notebook. The key factors here to consider are

1. Motion Correction has to be done on filtered imaging data. As there are low-frequency background activity here, they might be thought of as a pattern during motion correction leading to incorrect image registration. `caiman` has parameters that help with this.
2. In 2-photon, we assumed less number of background components. In this case, we have to use a different way of getting background estimates as they are not well behaved and has local variations. Therefore, a different backgroun model called the ring model is employed. Ring model estimates background at each pixel (yes! pixel level background estimation) by defining a ring around each pixel. This makes CNMFE more computationally expensive than CNMF.

## Preparing Data For Source Extraction

In [ ]:
fname = "data/data_endoscope.tif"
movie_orig = cm.load(fname)

In [ ]:
params = CNMFParams()

Change frame_rate to 10 and decay_time to 0.4 in the `data` part of params

In [ ]:
params.data.update(
    {
        'fr': 10,
        'decay_time': 0.4
    }
)

In [ ]:
motion_params = {
    'pw_rigid': False,
    'max_shifts': (5, 5),
    'strides': (48, 48),
    'overlaps': (24, 24),
    'max_deviation_rigid': 3
}
params.motion.update(motion_params)

mc = MotionCorrect(fname, **params.motion)
mc.motion_correct(save_movie=True)

plt.imshow(mc.total_template_rig, cmap='gray')
plt.show();

For 1-photon imaging data, it is important to remove low-frequency background activity. Therefore, we apply a gaussian filter as pre-processing step

**Example** Redo the motion correction with the same params as above with a modification to 'gSig_filt' = (3, 3)

In [ ]:
params.motion.update(
    {
        'gSig_filt': (3,3)
    }
)

mc = MotionCorrect(fname, **params.motion)
mc.motion_correct(save_movie=True)

plt.imshow(mc.total_template_rig, cmap='gray')
plt.show();

Redo the motion correction with 'gSig_filt' = (11, 11)

Let's exaggerate this even further! Redo the motion correction with 'gSig_filt' = (151, 151)

Great! Let's bring back the gaussian filter window to (3, 3) and finalize motion correction

Now, let's transform the memory mapped file into a format that can be used by CaImAn for source extraction

In [ ]:
mc_fname = mc.fname_tot_rig  
bord_px = 0  
fname_new = cm.save_memmap(mc_fname, base_name='memmap_', order='C', border_to_0=bord_px)

In [ ]:
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [ ]:
plt.imshow(np.max(images, axis=0), cmap='gray')
plt.show();

## CNMFE for 1-photon source extraction


There are two key parameters (along with a few more) that need to be set to trigger CNMFE algorithm instead of CNMF algorithm. 
They are:

1. nb = 0 (Number of background components. Setting this to 0 will make CaImAn use CNMFE algorithm)
2. method_init = 'corr_pnr' (Using correlation and Peak-to-Noise Ratio)


These are only to trigger CNMFE algorithm rather than CNMF. There are others (really) important parameters that we need to set At the end, we will consolidate these parameters and run CNMF fit.

In [ ]:
cnmfe_model = cnmf.CNMF(n_processes=1, params=params)

In [ ]:
cnmfe_model.params


**Example** Set 'method_init' to 'corr_pnr', 'nb' to 0, and 'center_psf' to True in params.

Inspect the image that opens in the browser. How many components did the model identify?

In [ ]:
cnmfe_model.params.change_params(
    {
        'nb': 0,
        'method_init': 'corr_pnr',
        'center_psf': True
    }
)
cnmfe_fit = cnmfe_model.fit(images)
corr_image = cm.local_correlations(images, swap_dim=False)
cnmfe_fit.estimates.plot_contours_nb(img=corr_image);

Let's try to set other parameters to identify more components. It's better to have false positives than false negatives as we can remove the bad components using component evaluation.

Set 'gSig' to np.array([3, 3]) and 'gSiz' to np.array([7, 7]). 'gSiz' is usually twice the value of 'gSig'+1

In [ ]:
cnmfe_model.params.change_params(
    {
        'gSig': np.array([3, 3]),
        'gSiz': np.array([7, 7])
    }
)
cnmfe_fit = cnmfe_model.fit(images)
corr_image = cm.local_correlations(images, swap_dim=False)
cnmfe_fit.estimates.plot_contours_nb(img=corr_image);

Let's remove the upper bound in K (number of components per patch) by making it `None` and set order of autoregressive (p) to 1. 

Optional: Downsample in time (`tsub`) by a factor of more than 2 and/or space (`ssub`) by a factor of more than 1 if memory issues come up

In [ ]:
cnmfe_model.params.change_params(
    {
        'K': None,
        'p': 1,
        'tsub': 2,
        'ssub': 1
    }
)
cnmfe_fit = cnmfe_model.fit(images)
corr_image = cm.local_correlations(images, swap_dim=False)
cnmfe_fit.estimates.plot_contours_nb(img=corr_image);

There are still many more parameters to be set here. Let's consolidate the important parameters that you can set or modify for 1-photon data 

Below explanation of each parameter is taken from the tutorial for CNMFE algorithm

| Parameter           | Description                                                               | Value                  |
|---------------------|---------------------------------------------------------------------------|------------------------|
| `p`                 | Order of the autoregressive system                                        | 1                      |
| `K`                 | Upper bound on number of components per patch, generally None for CNMFE   | None                   |
| `gSig`              | Expected half-width of neurons in pixels                                  | np.array([3, 3])       |
| `gSiz`              | Half-width of bounding box created around neurons during initialization   | 2*gSig + 1             |
| `merge_thr`         | Merging threshold, max correlation allowed                                | .7                     |
| `rf`                | Half-size of the patches in pixels (e.g., if rf=40, patches are 80x80)    | 40                     |
| `stride_cnmf`       | Amount of overlap between the patches in pixels                           | 20                     |
| `tsub`              | Downsampling factor in time for initialization                            | 2                      |
| `ssub`              | Downsampling factor in space for initialization                           | 1                      |
| `gnb`               | Number of background components (rank), set to 0 for CNMFE if positive    | 0                      |
| `low_rank_background` | Leaves background of each patch intact (use True if gnb>0)               | None                   |
| `nb_patch`          | Number of background components (rank) per patch (0 for CNMFE)            | 0                      |
| `min_corr`          | Min peak value from correlation image                                     | .8                     |
| `min_pnr`           | Min peak to noise ratio from PNR image                                    | 10                     |
| `ssub_B`            | Additional downsampling factor in space for background                    | 2                      |
| `ring_size_factor`  | Radius of ring is gSiz*ring_size_factor                                   | 1.4                    |


In [ ]:
cnmfe_model.params.change_params({
                                'method_init': 'corr_pnr',
                                'K': None,
                                'gSig': np.array([3,3]),
                                'gSiz': np.array([7,7]),
                                'p': 1,
                                'tsub': 2,
                                'ssub': 1,
                                'ssub_B': 2,
                                'nb': 0,
                                'nb_patch': 0,
                                'min_corr': 0.8,
                                'min_pnr': 10,
                                'center_psf': True,
                                'merge_thr': 0.7,
                                'low_rank_background': None,
                                'update_background_components': True,
                                'del_duplicates': True,
                                'rf': 40,
                                'stride': 20
})
cnmfe_fit = cnmfe_model.fit(images)
corr_image = cm.local_correlations(images, swap_dim=False)
cnmfe_fit.estimates.plot_contours_nb(img=corr_image);

There are still two parameters that may need modification. 

`min_corr` and `min_pnr` are **really** important parameters that control how many components are identified. 

Any pixel below `min_corr` value in the correlation image will be filtered out. Similarly, any pixels below `min_pnr` will also be filtered out. Therefore, setting these values as low as possible to retain most of the components is important

Set `min_corr` to 1.0 and `min_pnr` to 30. How many components are identified? 

Set `min_corr` to 0.5 and `min_pnr` to 15. How many components are identified? 

Let's use a vizualization that will help us set min_corr and min_pnr

In the resulting plot, adjust vmin of correlation image and PNR such that the images are mostly neurons and less of noise. The vmin values can be used as min_corr and min_pnr 

Remember that we need a lower bound and not the upper bound. Higher values will result in more false negatives

In [ ]:
gsig = (3, 3)
plt.figure()
correlation_image, peak_to_noise_ratio = cm.summary_images.correlation_pnr(images[::max(T//1000, 1)], 
                                                                           gSig=gsig[0], 
                                                                           swap_dim=False)

inspect_correlation_pnr(correlation_image, peak_to_noise_ratio)

In the below cell, change min_corr and min_pnr

In [ ]:
min_corr = 0.8
min_pnr = 12

cnmfe_model.params.change_params({
                                'method_init': 'corr_pnr',
                                'K': None,
                                'gSig': np.array([3,3]),
                                'gSiz': np.array([7,7]),
                                'p': 1,
                                'tsub': 2,
                                'ssub': 1,
                                'ssub_B': 2,
                                'nb': 0,
                                'nb_patch': 0,
                                'min_corr': #your_min_corr,
                                'min_pnr': #your_min_pnr,
                                'center_psf': True,
                                'merge_thr': 0.7,
                                'low_rank_background': None,
                                'update_background_components': True,
                                'del_duplicates': True,
                                'rf': 40,
                                'stride': 20
})
cnmfe_fit = cnmfe_model.fit(images)
corr_image = cm.local_correlations(images, swap_dim=False)
cnmfe_fit.estimates.plot_contours_nb(img=corr_image);

## Exploring the Estimates Class


**Example** How many components were identified here?

In [ ]:
cnmfe_fit.estimates.C.shape

**Example** Converting to pandas dataframe

In [ ]:
import pandas as pd

frame_rate = cnmfe_fit.params.data['fr']
frame_pd = 1/frame_rate
frame_times = np.linspace(0, T*frame_pd, T);

data_to_save = np.vstack((frame_times, cnmfe_fit.estimates.S)).T  # Transpose so time series are in columns
save_df = pd.DataFrame(data_to_save)
save_df.rename(columns={0:'time'}, inplace=True)
# save_df.to_csv('1p_spike_counts.csv')

**Example** Plot denoised calcium traces of the first component

In [ ]:
plt.figure()
plt.plot(cnmfe_fit.estimates.C[0])
plt.show();

**Caiman tutorials suggest to treat estimates.S (spike counts) with caution as there is no simultaneous ground-truth recordings to evaluate the deconvolution. Instead, they suggest to use estimates.C. Nevertheless, let's visualize them to see how they look like**

**Example** Plot spike counts of first component

In [ ]:
plt.figure()
plt.plot(cnmfe_fit.estimates.S[0])
plt.show();

**Example** Plot spike counts with spike times of first component

In [ ]:
plt.clf()
plt.plot(save_df['time'], save_df[1])
plt.xlabel('Time [s]')
plt.ylabel('Counts')
plt.show();

## Extra exercises

1. Set min_corr, min_pnr to very low and very high values. How does this affect the results?
2. Try changing other parameters to different values and see how it affects the results